In [ ]:
import glob
from sys import exit, path
from os.path import join, expanduser, exists

import numpy as np
import pandas as pd
import scipy.interpolate as spi
import scipy.signal as sps

from bokeh.io import output_file, export_png, export_svgs, show, output_notebook
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.models import ColorBar, ColumnDataSource, Span
from bokeh.layouts import gridplot
import bokeh.palettes
import colorcet as cc

path.insert(1, expanduser('~/src/noexiit/software/analyses'))

output_notebook()

In [ ]:
from analyze_fictrac import parse_dats
from analyze_fictrac import unconcat_df

pson_open_loop = parse_dats("/mnt/2TB/data_in/HK_20200317/pson_open_loop/", 1, 5, "offline")
dcor_open_loop = parse_dats("/mnt/2TB/data_in/HK_20200316/dcor_open_loop/", 1, 5, "offline")

In [ ]:
pson_open_loop_list = unconcat_df(pson_open_loop)
dcor_open_loop_list = unconcat_df(dcor_open_loop)

# Stimulus analyses

**Goal:** I have the beetle position relative to the ball map, and the ant position relative to the beetle position. What I want is to show the ant position relative to the ball map, as well as the beetle position relative to the ball map. I.e. I want to show beetle and ant movements relative to the same frame of reference.

In [ ]:
stims_pson_open_loop = sorted(glob.glob("/mnt/2TB/data_in/HK_20200317/pson_open_loop/*/stimulus/*.csv"))
stims_pson_open_loop_list = [pd.read_csv(stim) for stim in stims_pson_open_loop]

To work through a putative set of analyses, let's platy around with **one pair** of corresponding dataframes:

In [ ]:
stims_pson_open_loop_list[6].tail()

In [ ]:
pson_open_loop_list[6].tail()

In [ ]:
stim_df = stims_pson_open_loop_list[6]
fictrac_df = pson_open_loop_list[6]

We begin by interpolating from the 0 to 180 degrees range of the servo output, to real world values of 0 to 27 mm, which is the physical range of the servo output. We know this relationship is linear. Remember, this distance is the distance the servo, i.e. stimulus traveled. 

In [ ]:
# Generate interpolation function:
f_servo = spi.interp1d(np.linspace(0,180),np.linspace(0,27))

# Apply function:
stim_df["Servo output (mm)"] =  f_servo(stim_df["Servo output (degs)"])
stim_df.tail()

We next need to align our dataframes.
First, rename `Elapsed time` in the `stim_df` so it shares the column name `secs_elapsed` with the `fictrac_df`.

In [ ]:
stim_df = stim_df.rename(columns={"Elapsed time": "secs_elapsed"})

We now need to merge the two dataframes according to the common `secs_elapsed` column we just made. We are going to use the `merge_ordered` method, with the `fill_method` set to `ffill`, which means "forward fill". Forward fill takes care of `NaN` values by propagating the last valid observation forward.

In [ ]:
df_merged = pd.merge_ordered(stim_df, fictrac_df, on="secs_elapsed", fill_method="ffill")
df_merged.tail() 

But see how the last values are longer than acceptable propagations of the last valid observation from the shorter dataframe? 
Performing the forward fill propagated the _last_ valid observation for a longer than acceptable amount of time. We need to truncate the longer dataframe so that the entries on its last row is as close to the entries on the last row of the smaller dataframe. For example, if one dataframe recorded for 96 seconds, and the other for 100 seconds, then the forward fill method will propagate in the merged dataframe, the value from the 96th second all the way to the 100 second mark. That's 4 whole seconds of propagation, which is a long time, especially if our sampling frequency is much shorter than that. We handle this artifact with a simple function:

In [ ]:
def get_smaller_last_val_in_col(df1, df2, common_col):
    assert common_col in df1 and common_col in df2, \
        f"{df1} and {df2} do not share {common_col}"
    
    compare = float(df1[common_col].tail(1)) > float(df2[common_col].tail(1))
    if compare is True:
        return float(df2[common_col].tail(1))
    else:
        return float(df1[common_col].tail(1))

In [ ]:
smaller_last_val = get_smaller_last_val_in_col(stim_df, fictrac_df, "secs_elapsed")
smaller_last_val 

In [ ]:
# Drop anything larger than the smaller last val:
df_merged_trunc = df_merged[df_merged.secs_elapsed < smaller_last_val]
df_merged_trunc.tail()

Recall that the `servo output` is the amount the servo has extended from its initial position. In other words, by itself, `servo output` is not technically enough information to tell us how close or far the linear servo, i.e. ant stimulus, is from the beetle on the ball. For example, if the linear servo + ant tether was positioned very far from the beetle on the ball, then even at the linear servo's full extension, the ant will not touch the beetle. In other words, we have to make an approximation of how far the ant stimulus is from the beetle, when linear servo is fully extended. Making this measurement with both insects tethered is ... not practical, especially without preemptively releasing interaction behaviours. We instead have to make a simplifying assumption--that the full extension of the linear servo is approximately close enough to exactly make contact between the tethered beetle and the presented ant stimulus. From experimental observation, this assumption is reasonable, but the exact value will differ from trial to trial. I should think about a way of consistently measuring this distance so I that future experiments are more reproducible. 

If we assume the linear servo's full extension of 27 mm is when the ant exactly makes contact with the beetle, we can calculate how far the ant is from the tethered beetle:

\begin{equation}
\text{distance apart (mm)} = \text{27 mm} - \text{servo output (mm)}
\end{equation}

In [ ]:
df_merged_trunc["dist_from_stim"] = 27 - df_merged_trunc["Servo output (mm)"]
df_merged_trunc.tail(3)

My stepper motor homes to a reed switch and sets that home position as position 0. We can see from the dataset's initial observation below that the stepper position indeed starts at 0 degrees:

In [ ]:
df_merged_trunc.head(3)

From watching the FicTrac video, we can tell that when `stepper output (degs)` is 0, the beetle and ant are pointed in the same direction in true physical space. Therefore, the initial heading frame of reference of "0", is the same for both the beetle and the ant. In addition, `Stepper output (degs)` _**increases**_ in value when the stepper moves _**clockwise**_; clockwise here assumes that the observer is looking at the rig from the top-down. 

In [ ]:
# TODO: MAKE A MARKDOWN CELL EXPLAINING THE TRIG IN DETAIL

In [ ]:
df_merged_trunc['stim_X_mm'] = df_merged_trunc.apply(lambda row: (row["X_mm"] + (row["dist_from_stim"] * np.cos(np.deg2rad(row["Stepper output (degs)"])))), axis=1)
df_merged_trunc['stim_Y_mm'] = df_merged_trunc.apply(lambda row: (row["Y_mm"] + (row["dist_from_stim"] * np.sin(np.deg2rad(row["Stepper output (degs)"])))), axis=1)

We check that we get reasonable values for the stimulus X and Y coordinates:

In [ ]:
df_merged_trunc.loc[df_merged_trunc["Stepper output (degs)"] > 0.42].head(3)

In [ ]:
from analyze_stimulus import plot_fictrac_XY_with_stim

df_plot = df_merged_trunc
# df_plot = df_plot[::5]
# plot_fictrac_XY_with_stim(df_plot, high_percentile=97, alpha=0.06, size=2) 

## Alternative fill method: interpolation

Instead of doing a forward fill method where we propagate the last valid observation, we can make an interpolation of what the NaN values might be. 

In [ ]:
df_merged_no_fill = pd.merge_ordered(stim_df, fictrac_df, on="secs_elapsed", fill_method=None)

# Drop anything larger than the smaller last val:
df_merged_no_fill_trunc = df_merged_no_fill[df_merged_no_fill.secs_elapsed < smaller_last_val]
df_merged_no_fill_trunc.tail()

We use a linear interpolation. Understand that the beginning `NaN` values will not be interpolated, because those values are not preceded by valid observations, they are only proceeded by valid observations. In addition, non-numeric values will not be interpolated. We can fill those `NaN`s with a forward fill:

In [ ]:
df_merged_trunc_interp = df_merged_no_fill_trunc.interpolate()

In [ ]:
df_merged_trunc_interp.tail()

In [ ]:
df_merged_trunc_interp = df_merged_trunc_interp.ffill(axis=0)
df_merged_trunc_interp.tail()

We proceed with the rest of the pipeline:

In [ ]:
df_merged_trunc_interp["dist_from_stim"] = 27 - df_merged_trunc_interp["Servo output (mm)"]

df_merged_trunc_interp['stim_X_mm'] = df_merged_trunc_interp.apply(lambda row: (row["X_mm"] + (row["dist_from_stim"] * np.cos(np.deg2rad(row["Stepper output (degs)"])))), axis=1)
df_merged_trunc_interp['stim_Y_mm'] = df_merged_trunc_interp.apply(lambda row: (row["Y_mm"] + (row["dist_from_stim"] * np.sin(np.deg2rad(row["Stepper output (degs)"])))), axis=1)

df_merged_trunc_interp.tail()

In [ ]:
from analyze_stimulus import plot_fictrac_XY_with_stim

plot_fictrac_XY_with_stim(df_merged_trunc_interp)

## Test `analyze_stimulus.py` functions:

In [ ]:
from analyze_stimulus import parse_2dof_stimulus, merge_stimulus_with_data, make_stimulus_trajectory, plot_fictrac_XY_with_stim

# Use Platyusa open loop data as test:
stims = parse_2dof_stimulus("/mnt/2TB/data_in/HK_20200317/pson_open_loop/", 1, 0, 27, 27)
merged = merge_stimulus_with_data(stims, pson_open_loop, fill_method="linear")
stimmed = make_stimulus_trajectory(merged)

# Pull one out and plot:
stimmed_6 = unconcat_df(stimmed)[6]
plot_fictrac_XY_with_stim(stimmed_6, high_percentile=97, alpha=0.06, size=2) 